##### Importieren der notwendigen Bibliotheken:

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

##### Lesen der Verkaufsdaten

In [8]:
sales_df = pd.read_csv("../data/sales_clean.csv")
sales_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,SchoolHoliday,PublicHoliday,Easter,Christmas
0,1,5,2015-07-31,5263,555,1,1,1,0,0,0
1,2,5,2015-07-31,6064,625,1,1,1,0,0,0
2,3,5,2015-07-31,8314,821,1,1,1,0,0,0
3,4,5,2015-07-31,13995,1498,1,1,1,0,0,0
4,5,5,2015-07-31,4822,559,1,1,1,0,0,0


##### Erstellung neuer Metriken

Wir werden die Daten von 942 aggregieren, um individuelle Leistungsberichte für jede der Filialen zu erstellen. Die Spalten Date und DayOfWeek werden wir für diese Anlyse nicht mehr brauchen.

In [9]:
sales_df.drop(['Date', 'DayOfWeek'], axis=1, inplace=True)

Und nun gruppieren wir die Daten auf `Store` Ebene

In [10]:
global_sales_df = sales_df.groupby('Store').sum()
global_sales_df.head()

,Sales,Customers,Open,Promo,SchoolHoliday,PublicHoliday,Easter,Christmas
Store,,,,,,,,
1,3716854,440523,781,360,193,17,6,4
2,3883858,457855,784,360,167,15,6,4
3,5408261,584310,779,360,170,19,6,4
4,7556507,1036254,784,360,173,14,6,4
5,3642818,418588,779,360,172,21,6,4


Wir erstellen drei neue Variablen: `SalesPerDay`, `CustomersDay` und `SalesPerCustomer`.

In [11]:
global_sales_df['SalesPerDay'] = global_sales_df['Sales'] / global_sales_df['Open']
global_sales_df['CustomersPerDay'] = global_sales_df['Customers'] / global_sales_df['Open']
global_sales_df['SalesPerCustomer'] = global_sales_df['Sales'] / global_sales_df['Customers']

global_sales_df.head()

,Sales,Customers,Open,Promo,SchoolHoliday,PublicHoliday,Easter,Christmas,SalesPerDay,CustomersPerDay,SalesPerCustomer
Store,,,,,,,,,,,
1,3716854,440523,781,360,193,17,6,4,4759.096031,564.049936,8.437366
2,3883858,457855,784,360,167,15,6,4,4953.900510,583.998724,8.482725
3,5408261,584310,779,360,170,19,6,4,6942.568678,750.077022,9.255808
4,7556507,1036254,784,360,173,14,6,4,9638.401786,1321.752551,7.292138
5,3642818,418588,779,360,172,21,6,4,4676.274711,537.340180,8.702634


##### Verknüpfung mit den Daten der Filialen:

In [12]:
stores_df = pd.read_csv("../data/stores_clean.csv")
sales_stores_df = pd.merge(stores_df, global_sales_df, how = 'inner', on = 'Store')
sales_stores_df.set_index('Store', inplace=True)
sales_stores_df.head()

,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment,CompetitionDistance,COBefore2005-11-16,COBetween2005-11-16_2010-03-01,COAfter2010-03-01,Promo2,...,Customers,Open,Promo,SchoolHoliday,PublicHoliday,Easter,Christmas,SalesPerDay,CustomersPerDay,SalesPerCustomer
Store,,,,,,,,,,,,,,,,,,,,,
1,0,0,1,0,0,1270.0,0,1,0,0,...,440523,781,360,193,17,6,4,4759.096031,564.049936,8.437366
2,1,0,0,0,0,570.0,0,1,0,1,...,457855,784,360,167,15,6,4,4953.900510,583.998724,8.482725
3,1,0,0,0,0,14130.0,0,1,0,1,...,584310,779,360,170,19,6,4,6942.568678,750.077022,9.255808
4,0,0,1,0,2,620.0,0,1,0,0,...,1036254,784,360,173,14,6,4,9638.401786,1321.752551,7.292138
5,1,0,0,0,0,29910.0,0,0,1,0,...,418588,779,360,172,21,6,4,4676.274711,537.340180,8.702634


Betrachten wir die deskriptiven Statistiken für die Variablen, die direkt mit dem Gesamtumsatz zusammenhängen.

In [13]:
sales_vars = ['Sales', 'SalesPerDay', 'CustomersPerDay', 'SalesPerCustomer']
sales_stores_df[sales_vars].describe().apply(lambda x: x.apply('{0:.5f}'.format))

,Sales,SalesPerDay,CustomersPerDay,SalesPerCustomer
count,1115.00000,1115.00000,1115.00000,1115.00000
mean,5267426.56771,6934.20845,754.51016,9.64376
std,1951304.48397,2383.91105,353.34441,1.98686
min,2114322.00000,2703.73657,240.18310,3.51370
25%,3949377.00000,5322.29997,541.46869,8.13186
50%,4990259.00000,6589.94847,678.66752,9.46406
75%,6084147.50000,7964.20064,866.20330,10.98117
max,19516842.00000,21757.48342,3403.45860,16.16264


In der ersten Analyse hatten wir schon festgestellt, dass einige Filialen oft weit überdurchschnittliche Gewinne erzielen.  
Dies sind die Filialen, deren Gesamtumsatz mehr als 3 Standardabweichungen über dem Durchschnitt liegt.

In [14]:
sales_mean = sales_stores_df['Sales'].mean()
sales_std = sales_stores_df['Sales'].std()
high_sales = sales_mean + 3*sales_std

outlier_stores = sales_stores_df[sales_stores_df['Sales'] > high_sales].sort_values(by='Sales', ascending=False).index
sales_stores_df.iloc[outlier_stores]

,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment,CompetitionDistance,COBefore2005-11-16,COBetween2005-11-16_2010-03-01,COAfter2010-03-01,Promo2,...,Customers,Open,Promo,SchoolHoliday,PublicHoliday,Easter,Christmas,SalesPerDay,CustomersPerDay,SalesPerCustomer
Store,,,,,,,,,,,,,,,,,,,,,
263,1,0,0,0,2,1140.0,0,0,1,1,...,221342,622,286,124,21,6,2,3707.516077,355.855305,10.418606
818,0,0,0,1,0,490.0,1,0,0,1,...,496141,781,360,193,17,6,4,6110.572343,635.263764,9.618953
563,1,0,0,0,0,700.0,0,0,1,1,...,545236,776,360,170,19,6,4,5513.492268,702.623711,7.847006
1115,0,0,0,1,2,5350.0,1,0,0,1,...,337884,781,360,193,17,6,4,6302.469910,432.629962,14.567807
252,0,0,0,1,2,22330.0,1,0,0,1,...,630702,779,360,170,19,6,4,10615.512195,809.630295,13.111555
514,0,0,1,0,2,1200.0,0,0,1,1,...,375667,622,286,124,21,6,2,5756.009646,603.966238,9.530350
789,1,0,0,0,2,9770.0,1,0,0,0,...,364576,777,360,155,22,6,4,3380.011583,469.209781,7.203626
734,1,0,0,0,0,220.0,0,1,0,1,...,440954,779,360,170,19,6,4,5980.523748,566.051348,10.565338
384,1,0,0,0,2,130.0,1,0,0,1,...,781105,782,360,167,15,6,4,8871.575448,998.855499,8.881741
